In [35]:
import pygame,sys
from pygame.locals import *
import time
pygame.init()

FPS = 30
clock = pygame.time.Clock()

window_l = 500
window_w = 300
DISPLAYSURF = pygame.display.set_mode((window_l,window_w),0,32)
pygame.display.set_caption('Kill the blob!')

BLACK = (0,0,0)
WHITE = (255,255,255)
#img = pygame.image.load('Art//idle_r.png')
img_r = pygame.image.load('Art//idle_r.png')
#img_r = pygame.transform.rotozoom(img_r,0,2)
img_l = pygame.image.load('Art//idle_l.png')
#img_l = pygame.transform.rotozoom(img_l,0,2)
w_l = pygame.image.load('Art//walk_l.png')
#w_l = pygame.transform.rotozoom(w_l,0,2)
w_r = pygame.image.load('Art//walk_r.png')
#w_r = pygame.transform.rotozoom(w_r,0,2)
blb_l = pygame.image.load('Art//blob_l.png')
blb_r = pygame.image.load('Art//blob_r.png')
xpos = window_l/2
ypos = window_w-57


l_att = pygame.image.load('Art//attack_left.png')
#l_att = pygame.transform.rotozoom(l_att,0,2)
r_att = pygame.image.load('Art//attack_right.png')
#r_att = pygame.transform.rotozoom(r_att,0,2)

class Player:
    def __init__(self):
        self.xpos = xpos
        self.ypos = ypos
        self.img = img_r
        self.facingRight = True
        self.facingLeft = False
        self.ToObjRight = False
        self.ToObjLeft = True
        self.Flag=0
        self.attacking = False
        
    def walk(self,press):
        if press[K_LEFT] & self.facingRight== True:
            self.attacking=False
            self.facingLeft=True
            self.facingRight=False
            self.img = img_l
        elif press[K_LEFT] & self.facingLeft== True:
            if self.Flag==0:
                self.attacking=False
                self.img = w_l
                self.xpos = self.xpos-1
                self.Flag=1
                return
            elif self.Flag == 1:
                self.attacking=False
                self.img = img_l
                self.xpos = self.xpos-1
                self.Flag=0
                return
        elif press[K_RIGHT] & self.facingRight== True:
            if self.Flag==0:
                self.attacking=False
                self.img = w_r
                self.xpos = self.xpos+1
                self.Flag=1
                return
            elif self.Flag == 1:
                self.attacking=False
                self.img = img_r
                self.xpos = self.xpos+1
                self.Flag=0
                return
        elif press[K_RIGHT] & self.facingLeft== True:
            self.attacking=False
            self.facingLeft=False
            self.facingRight=True
            self.img = img_r
            
#    def jump(self,press):
#        if press[K_UP]:
#            for i in range(1000):
#                if i%500==0:
#                    self.ypos = self.ypos-1
#                    time.sleep(0.1)
#                    refresh()
#                    break
#            for i in range(1000):
#                if i%500==0:
#                    self.ypos = self.ypos+1
#                    time.sleep(0.1)
#                    refresh()
#                    break
#        else:
#            return
    
    def attack(self,blob,press):
        if press[K_SPACE]:
            if self.facingRight==True & self.ToObjLeft==True:
                self.attacking = True
                self.img = r_att
                if blob.blocking==False:
                    check_attack(self,blob)
            elif self.facingLeft==True & self.ToObjRight==True:
                self.attacking = True
                self.img = l_att
                if blob.blocking==False:
                    check_attack(self,blob)
        else:
            return

class FSMmng:
    def __init__(self,blob,Player):
        self.blob=blob
        self.Player = Player
        self.CurrentState = state_idle
    def update(self,blob,Player):
#        if self.CurrentState != None:
        self.CurrentState.Update(self.blob,self.Player)
    def ChangeState(self, newState):
        if self.CurrentState != None:
#            self.CurrentState.Exit()
            self.CurrentState = newState
#            self.CurrentState.Enter()
                    
class state_idle:
    def __init__(self,fsm):
        self.fsm = fsm
        self.NextState = None
#            def Enter(self):
#            def Exit(self):
    def Update(self,blob,Player) :
        if (Player.ToObjLeft == True) & ((blob.xpos-Player.xpos)>=80):
#            print("player on left far")
            blob.change_pos(Player.ToObjRight,Player.ToObjLeft)
        elif (Player.ToObjRight == True) & ((Player.xpos - (blob.xpos+100))>=30):
#            print("player on right far")
            blob.change_pos(Player.ToObjRight,Player.ToObjLeft)
        else :
            self.NextState = state_follow
            fsm.ChangeState(self.NextState)
class state_idle_adv:
    def __init__(self,fsm):
        self.fsm = fsm
        self.NextState = None
#            def Enter(self):
#            def Exit(self):
    def Update(self,blob,Player) :
        print('idle adv state')
        if (Player.ToObjLeft == True) & ((blob.xpos-Player.xpos)>=80):
            print("player on left far")
            blob.change_pos(Player.ToObjRight,Player.ToObjLeft)
        elif (Player.ToObjRight == True) & ((Player.xpos - (blob.xpos+100))>=30):
            print("player on right far")
            blob.change_pos(Player.ToObjRight,Player.ToObjLeft)
        else :
            self.NextState = state_follow_adv
            fsm.ChangeState(self.NextState)
class state_follow:
    def __init__(self,fsm):
        self.fsm = fsm
        self.NextState = None
#            def Enter(self):
#            def Exit(self):
    def Update(self,blob,Player):
        
        if Player.ToObjLeft == True & ((blob.xpos-Player.xpos)<80):
#            print('Player close on left started following')
            blob.follow_and_attack(Player)
            
        elif Player.ToObjRight == True & ((Player.xpos - (blob.xpos+100))<30):
#            print('Player close on right started following')
            blob.follow_and_attack(Player)
            
        else :
            self.NextState = state_idle
            fsm.ChangeState(self.NextState)
class state_follow_adv:
    def __init__(self,fsm):
        self.fsm = fsm
        self.NextState = None
#            def Enter(self):
#            def Exit(self):
    def Update(self,blob,Player):
        print('follow adv state',Player.ToObjLeft,Player.ToObjRight)
        blob.blocking==False
        if Player.attacking==False:
            
            if Player.ToObjLeft == True & ((blob.xpos-Player.xpos)<100):
                print('Player close on left started following')
                blob.follow_and_attack(Player)
            
            elif Player.ToObjRight == True & ((Player.xpos - (blob.xpos+100))<50):
                print('Player close on right started following')
                blob.follow_and_attack(Player)
            else :
                self.NextState = state_idle_adv
                fsm.ChangeState(self.NextState)
            
        elif Player.attacking==True:
            self.NextState=state_block
            fsm.ChangeState(self.NextState)
        
            
class state_block:
    def __init__(self,fsm):
        self.fsm = fsm
        self.NextState = None
    def Update(self,blob,Player):
        print('block state',Player.attacking)
        if Player.ToObjLeft == True & ((blob.xpos-Player.xpos)<80)&((blob.xpos-Player.xpos)>60) & Player.attacking==True:
            blob.block()
        elif Player.ToObjRight == True & ((Player.xpos - (blob.xpos+100))<30)&((Player.xpos - (blob.xpos+100))>10)&Player.attacking==True:
            blob.block()
        elif Player.attacking==False:
            self.NextState = state_follow_adv
            fsm.ChangeState(self.NextState)
        elif Player.ToObjLeft == True &((blob.xpos-Player.xpos)<60) & Player.attacking==True:
            self.NextState = state_follow_adv
            fsm.ChangeState(self.NextState)
        elif Player.ToObjRight == True&((Player.xpos - (blob.xpos+100))>10)&Player.attacking==True:
            self.NextState = state_follow_adv
            fsm.ChangeState(self.NextState)
class blob:
    def __init__(self):
        self.xpos = 360
        self.ypos = 194
        self.img = blb_l
        self.facingLeft = True
        self.facingRight = False
        self.count=0
        self.blocking=False
        
    def change_pos(self,ToObjRight,ToObjLeft):
#        print(ToObjRight,ToObjLeft)
        if ToObjLeft == True:
            self.img = blb_l
            self.facingLeft = True
            self.facingRight = False
        elif ToObjRight == True:
            self.img = blb_r
            self.facingLeft = False
            self.facingRight = True
    def follow_and_attack(self,Player):
        if Player.ToObjLeft == True:
            dif = Player.xpos+50 - self.xpos
            print(dif,self.xpos)
            if dif < 0:
                self.count=self.count+1
                print(self.count)
                if self.count==5:
                    self.xpos = self.xpos-1
                    self.count=0
            else:
                if Player.attacking != True:
                    Player.xpos = 1000
                run = False
                return
        elif Player.ToObjRight == True:
            dif = self.xpos+100 - Player.xpos
            print(dif,self.xpos)
            if dif <0:
                self.count = self.count+1
                if self.count==5:
                    self.xpos = self.xpos+1
                    self.count=0
            else:
                if Player.attacking != True:
                    Player.xpos = 1000
                run = False
                return
        else:
            return
    def block(self):
        self.blocking==True
    def movement(self,Player):
        pygame.time.delay(10)
        fsm.update(self,Player)

def check_attack(Player,blob):
    if Player.ToObjLeft==True & ((Player.xpos+50)>=blob.xpos):
        blob.xpos = 100
        fsm.ChangeState(state_idle_adv)
    if Player.ToObjRight==True & (Player.xpos<=(blob.xpos+100)):
        blob.xpos = 100
        fsm.ChangeState(state_idle_adv)
def refresh():
    DISPLAYSURF.fill(WHITE)
    pd = DISPLAYSURF.blit(Player.img,(Player.xpos,Player.ypos))
    bd = DISPLAYSURF.blit(blob.img,(blob.xpos,blob.ypos))
    if pd.right < bd.left:
        Player.ToObjLeft = True
        Player.ToObjRight = False
    elif pd.left > bd.right:
        Player.ToObjLeft = False
        Player.ToObjRight = True
    blob.movement(Player)
    
run = True
Player = Player()
blob = blob()
fsm = FSMmng(blob,Player)
state_idle = state_idle(fsm)
state_follow = state_follow(fsm)
state_idle_adv = state_idle_adv(fsm)
state_follow_adv = state_follow_adv(fsm)
state_block = state_block(fsm)
fsm.ChangeState(state_idle)
#pygame.key.set_repeat(10,10)
while(run):
    refresh()
    
    press = pygame.key.get_pressed()
    Player.walk(press)
    Player.attack(blob,press)
    
    pygame.time.delay(10)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    
        pygame.display.update()
        clock.tick(FPS)

-28.0 360
1
-27.0 360
2
-27.0 360
3
-27.0 360
4
-27.0 360
5
-26.0 359
1
-25.0 359
2
-24.0 359
3
-23.0 359
4
-22.0 359
5
-21.0 358
1
-21.0 358
2
-21.0 358
3
-21.0 358
4
-21.0 358
5
-19.0 357
1
-18.0 357
2
-18.0 357
3
-18.0 357
4
-18.0 357
5
-17.0 356
1
-17.0 356
2
-16.0 356
3
-15.0 356
4
-14.0 356
5
-13.0 355
1
-13.0 355
2
-13.0 355
3
-13.0 355
4
-13.0 355
5
-11.0 354
1
-10.0 354
2
-9.0 354
3
-8.0 354
4
-8.0 354
5
-7.0 353
1
-7.0 353
2
-7.0 353
3
-7.0 353
4
-7.0 353
5
-6.0 352
1
-6.0 352
2
-6.0 352
3
-6.0 352
4
-6.0 352
5
-5.0 351
1
-5.0 351
2
-5.0 351
3
-5.0 351
4
-5.0 351
5
-4.0 350
1
-4.0 350
2
-4.0 350
3
-4.0 350
4
-4.0 350
5
-3.0 349
1
-3.0 349
2
-3.0 349
3
-3.0 349
4
-3.0 349
5
-2.0 348
1
-2.0 348
2
-2.0 348
3
-2.0 348
4
-2.0 348
5
-1.0 347
1
-1.0 347
2
-1.0 347
3
-1.0 347
4
-1.0 347
5
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player o

-12.0 112
follow adv state False True
Player close on right started following
-11.0 113
follow adv state False True
Player close on right started following
-11.0 113
follow adv state False True
Player close on right started following
-11.0 113
follow adv state False True
Player close on right started following
-11.0 113
follow adv state False True
Player close on right started following
-11.0 113
follow adv state False True
Player close on right started following
-10.0 114
follow adv state False True
Player close on right started following
-10.0 114
follow adv state False True
Player close on right started following
-10.0 114
follow adv state False True
Player close on right started following
-10.0 114
follow adv state False True
Player close on right started following
-10.0 114
follow adv state False True
Player close on right started following
-9.0 115
follow adv state False True
Player close on right started following
-9.0 115
follow adv state False True
Player close on right starte

player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far
idle adv state
player on right far


SystemExit: 

C:\Users\Joy Parikh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
